In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [12]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [13]:
x

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [15]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [16]:
x.shape

(768, 8)

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=1)

In [115]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [20]:
model = Sequential()
model.add(Dense(32,activation="relu",input_dim = 8))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                288       
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(x_train,y_train,batch_size = 32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4449 - accuracy: 0.7769 - val_loss: 0.4723 - val_accuracy: 0.7727
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4436 - accuracy: 0.7801 - val_loss: 0.4728 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4428 - accuracy: 0.7801 - val_loss: 0.4720 - val_accuracy: 0.7727
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4425 - accuracy: 0.7785 - val_loss: 0.4723 - val_accuracy: 0.7727
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4415 - accuracy: 0.7785 - val_loss: 0.4736 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4402 - accuracy: 0.7785 - val_loss: 0.4732 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4397 - accuracy: 0.7801 - val_loss: 0.4723 - val_accuracy: 0.7792
Epoch 8/100
2

* 1. How to select appropriate optimizer
* 2. No. of nodes in a layer
* 3. How to select no. of layers
* 4. All in all one model

In [26]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


##  How to select appropriate optimizer

In [29]:
import keras_tuner as kt

In [49]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation="relu",input_dim = 8))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd",'rmsprop','adadelta']),
                loss="binary_crossentropy",metrics=["accuracy"])
  return model

In [52]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [53]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.8051947951316833

Trial 3 summary
Hyperparameters:
optimizer: adam
Score: 0.7272727489471436

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.6623376607894897

Trial 1 summary
Hyperparameters:
optimizer: sgd
Score: 0.6558441519737244


In [55]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [56]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
model.optimizer.get_config()

{'name': 'RMSprop',
 'weight_decay': None,
 'clipnorm': None,
 'global_clipnorm': None,
 'clipvalue': None,
 'use_ema': False,
 'ema_momentum': 0.99,
 'ema_overwrite_frequency': 100,
 'jit_compile': False,
 'is_legacy_optimizer': False,
 'learning_rate': 0.001,
 'rho': 0.9,
 'momentum': 0.0,
 'epsilon': 1e-07,
 'centered': False}

In [57]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 19ms/step - loss: 0.5350 - accuracy: 0.7394 - val_loss: 0.5177 - val_accuracy: 0.8247
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5218 - accuracy: 0.7476 - val_loss: 0.5065 - val_accuracy: 0.7987
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5105 - accuracy: 0.7541 - val_loss: 0.4958 - val_accuracy: 0.7922
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5009 - accuracy: 0.7590 - val_loss: 0.4887 - val_accuracy: 0.7857
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4933 - accuracy: 0.7655 - val_loss: 0.4824 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4869 - accuracy: 0.7704 - val_loss: 0.4765 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4809 - accuracy: 0.7736 - val_loss: 0.4720 - val_accuracy: 0.7792
Epoch 8/100


## Number of nodes in a layers

In [66]:
def build_model(hp):

  model = Sequential()

  units = hp.Int("units",min_value = 8,max_value = 128,step = 8) # starting node size 8 after that 16,24...
  model.add(Dense(units,activation="relu",input_dim = 8))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer="rmsprop",loss='binary_crossentropy',metrics=["accuracy"])

  return model

In [70]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=5,
                        directory="mydir",
                        project_name="Number of nodes")

In [71]:
tuner.search(x_train,y_train,epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [72]:
tuner.results_summary()

Results summary
Results in mydir/Number of nodes
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units: 48
Score: 0.798701286315918

Trial 0 summary
Hyperparameters:
units: 56
Score: 0.7922077775001526

Trial 4 summary
Hyperparameters:
units: 80
Score: 0.7922077775001526

Trial 3 summary
Hyperparameters:
units: 88
Score: 0.7857142686843872

Trial 1 summary
Hyperparameters:
units: 16
Score: 0.7727272510528564


In [77]:
tuner.get_best_hyperparameters()[0].values

{'units': 48}

In [73]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                432       
                                                                 
 dense_1 (Dense)             (None, 1)                 49        
                                                                 
Total params: 481 (1.88 KB)
Trainable params: 481 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
model.get_compile_config()

{'optimizer': 'rmsprop',
 'loss': 'binary_crossentropy',
 'metrics': ['accuracy'],
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

In [78]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4538 - accuracy: 0.7769 - val_loss: 0.4467 - val_accuracy: 0.8117
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4521 - accuracy: 0.7834 - val_loss: 0.4461 - val_accuracy: 0.8182
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4505 - accuracy: 0.7818 - val_loss: 0.4445 - val_accuracy: 0.8247
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4499 - accuracy: 0.7769 - val_loss: 0.4448 - val_accuracy: 0.8247
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4482 - accuracy: 0.7752 - val_loss: 0.4460 - val_accuracy: 0.8247
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4476 - accuracy: 0.7720 - val_loss: 0.4455 - val_accuracy: 0.8247
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4461 - accuracy: 0.7752 - val_loss: 0.4460 - val_accuracy: 0.8117
Epoch 8/100
2

## How to select number of layers

In [83]:
def build_model(hp):
  model  = Sequential()

  model.add(Dense(48,activation='relu',input_dim=8))

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)): # number of layers
    model.add(Dense(48,activation="relu"))

  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer="rmsprop",loss='binary_crossentropy',metrics=["accuracy"])
  return model


In [84]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory="mydirr",
                        project_name="num_layers")

In [85]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 16s


In [86]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [87]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                432       
                                                                 
 dense_1 (Dense)             (None, 48)                2352      
                                                                 
 dense_2 (Dense)             (None, 48)                2352      
                                                                 
 dense_3 (Dense)             (None, 48)                2352      
                                                                 
 dense_4 (Dense)             (None, 48)                2352      
                                                                 
 dense_5 (Dense)             (None, 48)                2352      
                                                                 
 dense_6 (Dense)             (None, 1)                 4

In [88]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4593 - accuracy: 0.7736 - val_loss: 0.4728 - val_accuracy: 0.8052
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4489 - accuracy: 0.7948 - val_loss: 0.4772 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4420 - accuracy: 0.7834 - val_loss: 0.4630 - val_accuracy: 0.8247
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4375 - accuracy: 0.7915 - val_loss: 0.4614 - val_accuracy: 0.7857
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4295 - accuracy: 0.7964 - val_loss: 0.4663 - val_accuracy: 0.8052
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4206 - accuracy: 0.8078 - val_loss: 0.4604 - val_accuracy: 0.8247
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4158 - accuracy: 0.7997 - val_loss: 0.4642 - val_accuracy: 0.7987
Epoch 8/100


## All in one

In [116]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)): # number of layers

    if counter == 0:
      model.add(Dense(hp.Int("units" + str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice("activation" + str(i),values=["relu","tenh","sigmoid"]),
                      input_dim = 8))
      model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int("units" + str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice("activation" + str(i),values=["relu","tenh","sigmoid"])))
      model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter += 1

  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd",'rmsprop','adadelta','nadam']),
                loss="binary_crossentropy",metrics=["accuracy"])
  return model



In [117]:
tuner =  kt.RandomSearch(build_model,
                    objective="val_accuracy",
                    max_trials=5,
                    directory="mydir",
                    project_name="Final")

In [118]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6558441519737244

Best val_accuracy So Far: 0.6623376607894897
Total elapsed time: 00h 00m 12s


In [119]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 72,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'adadelta',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1}

In [120]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dropout (Dropout)           (None, 72)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 584       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 1)                 9

In [121]:
model.get_compile_config()

{'optimizer': 'adadelta',
 'loss': 'binary_crossentropy',
 'metrics': ['accuracy'],
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

In [122]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 16ms/step - loss: 0.6828 - accuracy: 0.6303 - val_loss: 0.6884 - val_accuracy: 0.6623
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6813 - accuracy: 0.6270 - val_loss: 0.6883 - val_accuracy: 0.6623
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6855 - accuracy: 0.6173 - val_loss: 0.6883 - val_accuracy: 0.6623
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6856 - accuracy: 0.6107 - val_loss: 0.6882 - val_accuracy: 0.6623
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6856 - accuracy: 0.5928 - val_loss: 0.6882 - val_accuracy: 0.6623
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6800 - accuracy: 0.6270 - val_loss: 0.6882 - val_accuracy: 0.6623
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6818 - accuracy: 0.6091 - val_loss: 0.6881 - val_accuracy: 0.6623
Epoch 8/100


In [147]:
# https://rasbt.github.io/mlxtend/user_guide/plotting/plot_decision_regions/